In [ ]:
import numpy as np
import xarray as xr

In [ ]:
time = xr.DataArray(
    np.linspace(0, 10),
    dims=("time"),
    attrs={"units":"s"},
    name="time",
)
time.plot()

In [ ]:
# see https://stackoverflow.com/a/59378218/8863304
dt = xr.DataArray(
    np.diff(time, prepend=0),
    coords=[time],
    attrs={"units":"s"},
    name="delta_time"
)
dt

In [ ]:
def gen_random_array(*, shape, noise, center):
    """Create an array with values between -0.5 and 0.5."""
    rnd_array = np.random.default_rng().normal(size=shape)
    offset = -0.5
    center = np.array(center)
    return ((rnd_array + offset) * noise) + center

axis = xr.DataArray(
    ["x", "y"],
    dims=("axis"),
    name="axis"
)


acceleration = xr.DataArray(
    gen_random_array(
        shape=(len(time), len(axis)),
        noise=0.8,
        center=[0.0, -9.81],
    ),
    coords=[time, axis],
)

acceleration.plot(hue="axis")

In [ ]:
simulation = xr.Dataset(
    {
        "time":time,
        "axis":axis,
        "acceleration":acceleration,
    }
)
simulation

In [ ]:
simulation = xr.Dataset(
    {
        "acceleration":acceleration,
    },
)
simulation

In [ ]:
simulation = xr.Dataset(
    {
        "acceleration":acceleration,
    },
    coords={
        "time":time,
        "axis":axis,
    }
)
simulation

In [ ]:
initial_velocity = np.array([5.0, 40.0])

simulation["velocity"] = (dt * simulation.acceleration).cumsum() + initial_velocity

simulation.velocity.plot(hue="axis")

In [ ]:
initial_position = np.array([0.0, 40.0])

simulation["position"] = (dt * simulation.velocity).cumsum() + initial_position

simulation.position.plot(hue="axis")

In [ ]:
simulation.position.where(
    simulation.p
)

## Dimensions / Coordinates

In [ ]:
time_values = np.linspace(0, 10)

x_values = np.arange(0, 20)
y_values = np.arange(0, 20)

time_da = xr.DataArray(
    time_values,
    dims=("time"),
    attrs={"units":"s"},
    name="time",
)

x_da = xr.DataArray(
    x_values,
    dims=("x"),
    attrs={"units":"m"},
    name="x",
)

y_da = xr.DataArray(
    y_values,
    dims=("y"),
    attrs={"units":"m"},
    name="y",
)

In [ ]:
base_temperature = 20.0

temperature_values = np.zeros((
    len(time_values), len(x_values), len(y_values)
))

temperature_values.fill(base_temperature)

temperature_da = xr.DataArray(
    temperature_values,
    coords=[time_da, x_da, y_da],
    name="temperature"
)

mask = (
    (temperature_da.time == 0)
    & (temperature_da.x <= 5)
    & (temperature_da.y <= 5)
)

initial_temperature = 1000

temperature_da = xr.where(
    mask,
    initial_temperature,
    temperature_da
)

temperature_da.plot.hist()

In [ ]:
y_da.plot()